In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
# from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import re
from nltk.stem import WordNetLemmatizer


In [2]:
robotics = pd.read_csv("data/robotics.csv")
cooking = pd.read_csv("data/cooking.csv")
biology = pd.read_csv("data/biology.csv")
crypto = pd.read_csv("data/crypto.csv")
travel = pd.read_csv("data/travel.csv")
diy = pd.read_csv("data/diy.csv")             
test = pd.read_csv("data/test.csv") 

df_hash = {
    "cooking": cooking,
    "crypto": crypto,
    "robotics": robotics,
    "biology": biology,
    "travel": travel,
    "diy":diy,
    "test": test
    
}

print(df_hash["robotics"].iloc[0])
print df_hash["robotics"].iloc[1]
print(df_hash["robotics"].iloc[2])
# print df_hash["biology"].head(1)

id                                                         1
title      What is the right approach to write the spin c...
content    <p>Imagine programming a 3 wheel soccer robot....
tags                                          soccer control
Name: 0, dtype: object
id                                                         2
title      How can I modify a low cost hobby servo to run...
content    <p>I've got some hobby servos (<a href="http:/...
tags                                         control rcservo
Name: 1, dtype: object
id                                                         3
title      What useful gaits exist for a six legged robot...
content    <p><a href="http://www.oricomtech.com/projects...
tags                                               gait walk
Name: 2, dtype: object


In [3]:
print df_hash['robotics'].columns.values
print df_hash['test'].head(5)

['id' 'title' 'content' 'tags']
   id                                              title  \
0   1  What is spin as it relates to subatomic partic...   
1   2  What is your simplest explanation of the strin...   
2   3   Lie theory, Representations and particle physics   
3   7                 Will Determinism be ever possible?   
4   9                               Hamilton's Principle   

                                             content  
0  <p>I often hear about subatomic particles havi...  
1  <p>How would you explain string theory to non ...  
2  <p>This is a question that has been posted at ...  
3  <p>What are the main problems that we need to ...  
4  <p>Hamilton's principle states that a dynamic ...  


In [4]:
 for topic, df in df_hash.iteritems():
    print topic + ": " +  str(len(df.index))

biology: 13196
cooking: 15404
travel: 19279
robotics: 2771
crypto: 10432
diy: 25918
test: 81926


In [5]:
import nltk
# nltk.download()
from nltk.corpus import stopwords
# In Python, searching a set is much faster than searching a list, so convert the stop words to a set
stops = set(stopwords.words("english"))  

https://www.kaggle.com/c/word2vec-nlp-tutorial#part-1-for-beginners-bag-of-words

In [6]:
def separate_tags(tags):
#     print tags
#     letters_only = re.sub("[^a-zA-Z]", " ", tags) 
#     return map( lambda x: [x], letters_only.lower().split() )
    return tags.split(" ")

# print separate_tags("hello googbye13 42 HI")

In [7]:
def strip_html(raw_html):
    if raw_html:
        # remove html tags
        soup = BeautifulSoup(raw_html, "html.parser")

        [s.extract() for s in soup(['pre', 'code'])]
            
        question_text = soup.get_text()
        
        # remove everything but letters
        letters_only = re.sub("[^a-zA-Z]", " ", question_text) 
        
            
        # normalize case
        words = letters_only.lower().split()   

        # remove stopwords         
        meaningful_words = [w for w in words if not w in stops] 
        
        wordnet_lemmatizer = WordNetLemmatizer()
        meaningful_word_stems = map(lambda x: wordnet_lemmatizer.lemmatize(x) , meaningful_words)
        return( " ".join( meaningful_word_stems )) 
    else:
        return ""
    
# strip_html(df_hash['robotics'].content[0])

In [8]:
print "Cleaning and parsing the training set of tags for each topic...\n"

for topic, df in df_hash.iteritems():
    if topic == "test":
        next
    else:
        print topic
        print "'s size is "

        df.content = df.content.apply(lambda x: strip_html(x) )
        df.title = df.title.apply(lambda x: strip_html(x) )
        df.tags = df.tags.apply(lambda y: separate_tags(y) )
        df.tags.replace(['', ' ', 'untagged'], np.nan, inplace=True)
        df.dropna(subset=['tags'], inplace=True)
        print df.tags
        print df.content.head(4)

Cleaning and parsing the training set of tags for each topic...

biology
's size is 
0        [ribosome, binding-sites, translation, synthet...
1                                      [rna, biochemistry]
2                   [immunology, cell-biology, hematology]
3                                           [cell-culture]
4            [splicing, mrna, spliceosome, introns, exons]
5                   [dna, biochemistry, molecular-biology]
6                                 [neuroscience, synapses]
7                                               [plasmids]
8        [molecular-genetics, gene-expression, experime...
9                  [evolution, mitochondria, chloroplasts]
10                           [high-throughput, cell-based]
11                  [molecular-biology, synthetic-biology]
12                              [bioinformatics, homework]
13                              [neuroscience, immunology]
14                                     [splicing, histone]
15                      [genom

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

for topic, df in df_hash.iteritems():
    if topic == "test":
        next
    else:
        vectorizer = TfidfVectorizer(analyzer='word', tokenizer=lambda doc: doc, lowercase=False , stop_words="english" ,min_df=1 ) 
        print topic
        X = vectorizer.fit_transform(df.title)
        indices = np.argsort(vectorizer.idf_)[::-1]
        features = np.array(vectorizer.get_feature_names())
        top_n = 10
        top_features = [features[i] for i in indices[:top_n]]
        print top_features

        for top_feat_index in indices[:15]:
            print features[top_feat_index]
            print vectorizer.idf_[top_feat_index]

biology
[u'j', u'z', u'q', u'x', u'k', u'w', u'v', u'f', u'b', u'y']
j
5.13588667654
z
3.81192158838
q
3.57400745936
x
3.00618910811
k
2.75581408807
w
2.58933371721
v
2.19444545131
f
1.94976907942
b
1.88713001046
y
1.8024980336
h
1.72588143674
g
1.5484260696
p
1.46409552735
u
1.45163878465
m
1.395737625
cooking
[u'j', u'q', u'x', u'z', u'v', u'w', u'y', u'f', u'b', u'h']
j
4.26572046313
q
4.02370842751
x
3.81157317638
z
3.5391253485
v
2.35693410195
w
2.23797508968
y
2.12705584156
f
1.83587359053
b
1.81424632186
h
1.67660191628
k
1.67074501133
m
1.61759359919
p
1.60455747605
d
1.4738669733
g
1.46970506947
travel
[u'q', u'j', u'z', u'x', u'w', u'f', u'k', u'b', u'y', u'm']
q
3.92370114535
j
3.80463193588
z
3.49344925825
x
3.35475232233
w
2.20397280433
f
2.03379826088
k
1.96777513428
b
1.88526762739
y
1.76040030961
m
1.70200371149
v
1.63433039155
h
1.60085929244
p
1.47041865312
d
1.39889957156
g
1.39612093439
robotics
[u'z', u'j', u'x', u'q', u'k', u'w', u'y', u'v', u'f', u'h']
z
3.809330

In [10]:
print df_hash["robotics"].shape



(2771, 4)


In [11]:
# from sklearn.base import BaseEstimator, TransformerMixin

# class QuestionStripper(BaseEstimator, TransformerMixin):

#     def __init__(self):
#         self = self
#         self.df = df  # e.g. pass in a column name to extract
#         self.topic_size = df.size
        
#     def transform(self, X, y=None):
#         return do_something_to(X, self.vars)  # where the actual feature extraction happens
#         print "'s size is "
#         print self.topic_size
#         return X.apply(lambda x: strip_html(x) )
        
    
#     def fit(self, X, y=None):
#         return self  # generally does nothing

In [12]:
print df_hash["robotics"].tags.head(5)
print df_hash["robotics"].shape
print df.content.head(4)

0                           [soccer, control]
1                          [control, rcservo]
2                                [gait, walk]
3    [microcontroller, arduino, raspberry-pi]
4                      [motion-planning, rrt]
Name: tags, dtype: object
(2771, 4)
0    <p>I often hear about subatomic particles havi...
1    <p>How would you explain string theory to non ...
2    <p>This is a question that has been posted at ...
3    <p>What are the main problems that we need to ...
Name: content, dtype: object


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold, cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, f1_score, make_scorer, accuracy_score, confusion_matrix
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OutputCodeClassifier, OneVsRestClassifier, OneVsOneClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from math import sqrt
from skmultilearn.ensemble import RakelD, RakelO
from skmultilearn.problem_transform import BinaryRelevance, LabelPowerset
from sklearn.naive_bayes import MultinomialNB

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# from sklearn.feature_selection import SelectFromModel
from time import time
from skmultilearn.problem_transform import BinaryRelevance  
# from nltk.classify.scikitlearn import SklearnClassifier
# from collections import defaultdict
# from sklearn.linear_model import LogisticRegressionCV

from sklearn.cluster import MiniBatchKMeans
# parameters = {
#     'rakel__labelset_size': range(1, 3),
#     'classifier': [LabelPowerset(), SVC()],
#     'rakel__classifier': "classifier"
#     }

# parameters = {'tree__max_depth': range(7, 20),
#               "tree__n_estimators": [10, 50, 100],
#               "tree__bootstrap": [ False],
#               "tree__criterion": ["gini", "entropy"]}
# parameters = {
#     'rakel__labelset_size': range(1, 3),
#     'vectorizer__ngram_range': range(1,2),
#     'rakel__classifier': [LabelPowerset(), BinaryRelevance()],
#     'rakel__classifier__classifier': [MultinomialNB()],
#     'rakel__classifier__classifier__alpha': [0.7, 1.0],
# }

# TfidfTransformer = TfidfVectorizer(analyzer = "word",   \

        

# use a full grid over all parameters
# param_grid = {"max_depth": [3, None],
#               "max_features": [1, 3, 10],





# Set the parameters by cross-validation
# parameters = {'svc__estimator__kernel': ['rbf', 'poly']}
#               'svc__estimator__degree':[1, 2, 3] }

# parameters = dict(svc__estimator__C=np.logspace(-5,0,5), svc__estimator__gamma=np.logspace(-2, 2, 10))

#     print("Best parameters set found on development set:")
#     print()
#     print(clf.best_params_)
#     print()
#     print("Grid scores on development set:")

        
df = df_hash["robotics"]
# for topic, df in df_hash.iteritems():
print "splitting"


#     tf = TfidfVectorizer()
#     X = tf.fit_transform(df.content.values)
#     Y = df.tags.values
print "len df.tags"
print len(df.tags)
print df.tags.shape
print df.tags.head(5)
mlb = MultiLabelBinarizer()
Y = pd.DataFrame( mlb.fit_transform(df.tags) )

print "y shape"
print Y.shape
print "y"
print Y.head(5)
print type(Y)
true_k = len(Y)
print "true K: \n"
print int(sqrt(true_k))
print int(true_k/100)
# parameters = {
#         "rf_classifier__n_estimators" : [ 5 , 10],
#     "dt_class__max_depth" : [20,50, 100, 500 ],
#     "dt_class__criterion" : ["gini", "entropy"]
# } 

#     steps = [   ('vectorizer', CountVectorizer(stop_words="english", strip_accents='unicode')),
steps = [('tfidf', TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False , analyzer = "word",
                                   stop_words="english",max_features = 5000 ) ),
#         ('rakel', RakelD(MultinomialNB(alpha=0.7), labelset_size=true_k))]
#             ('rf_classifier', RandomForestClassifier(n_estimators= 7, max_depth=20, random_state=42 ))]
            ("dt_class", OneVsRestClassifier(DecisionTreeClassifier( random_state = 42)))]
#             ( 'kmeans_cluster', MiniBatchKMeans(n_clusters=int(sqrt(true_k)), init='k-means++', n_init=1,
#                              init_size=1000, batch_size=1000) ) ]
#             ('svc', OneVsRestClassifier())]

pipeline = Pipeline(steps)
#     Y = mlb.fit_transform(df.tags)z
#     print Y.shape
#     print Y
#     print df.tags.head(5)
#     X_train, X_test, y_train, y_test_vect = train_test_split(df.content,\
#                                                         Y, \
#                                                         test_size=0.33,\
#                                                         random_state=42)



#     scorer = make_scorer(f1_score, average="weighted")
#     Y_test = mlb.fit_transform(y_test)
#     scores = ['precision', 'recall']

#     for score in scores:
#     print("# Tuning hyper-parameters for %s" % score)
#     cv = GridSearchCV( pipeline, scoring='f1')
#     cv = GridSearchCV( pipeline, param_grid = parameters, scoring='f1')
#                            scoring='%s_macro' % score)



t0 = time()
# clf = GridSearchCV(pipeline, parameters, cv=10)
# clf.fit( df.content, Y )
#     pipeline.fit(X_train, y_train)
#     y_predicted_vect = pipeline.predict(X_test)
print("done in %0.3fs" % (time() - t0))


ImportError: No module named skmultilearn.ensemble

In [ ]:

# print clf.best_estimator_
# print("Best score: %0.3f" % clf.best_score_)
# print("Best parameters set:")
# best_parameters = clf.best_estimator_.get_params()
# print best_parameters
# for param_name in sorted(parameters.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name]))

# print clf.cv_results_.keys()
# print("Best score: %0.3f" % clf.best_score_)
#     results = cross_val_score(clf, X, Y)
#     print results
#     y_prediction = clf.predict( X_test )


In [ ]:
# for topic, df in df_hash.iteritems():
print "evaluating"
t0 = time()
y_prediction =  cross_val_predict(pipeline, df.content, Y, cv=10 )
print("done in %0.3fs" % (time() - t0))
print "y_prediction: \n"
print y_prediction.shape
print y_prediction
print type(y_prediction)
print type(y_prediction[0])
print y_prediction.astype(int)
#     print y_predicted_vect.to_dense

Y_inv = mlb.inverse_transform(y_prediction)
#     y_test_inv = mlb.inverse_transform(y_test_vect.values)
#     y_pred_inv = mlb.inverse_transform(y_predicted_vect)
#     y_pred_inv = y_predicted_vect.apply(lambda x: d[x].inverse_transform(x))
#     y_test_inv = y_test_vect.apply(lambda x: d[x].inverse_transform(x))

#     print clf.best_estimator_
print "baallss"
#     print Y_inv
print accuracy_score(Y, y_prediction) 
report = classification_report(Y, y_prediction) 
print(report)

#     print cv.best_score_
#     print cv.best_params_
#     print clf.feature_importances_  
#     model = SelectFromModel(clf, prefit=True)
#     print model





In [ ]:
# y = mlb.inverse_transform(Y)
index = 0
for actual, pred in zip(df.tags, Y_inv):
        print('index: {0} :: {1} => {2}'.format(index, actual, ', '.join(pred)))
        index = index +1

        

In [ ]:
print y_prediction.shape
# print y_test[0]

In [ ]:
# print confusion_matrix(Y, y_prediction) 
# print y_pred_inv
# print mlb.inverse_transform(0)


https://www.civisanalytics.com/blog/workflows-in-python-using-pipeline-and-gridsearchcv-for-more-compact-and-comprehensive-code/

In [ ]:
# Numpy arrays are easy to work with, so convert the result to an 
# array
# train_data_features = train_data_features.toarray()
# report = classification_report(Y, y_prediction) 
# print report
f1_score(Y, y_prediction, average='weighted')

In [ ]:
# print train_data_features.shape
# print "4"
